# Artificial Neural Network

Here I implemented a quick Keras model to work on this dataset. 
The idea was to see how high of a accuracy I could get.


Deep Learning works better with bigger datasets, 
this one being really tiny. 
Yet I could get some good results very quickly.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from subprocess import check_output

In [3]:
dataset = pd.read_csv('HR_Attrition.csv')

## Cleaning the data

Removing the following columns:

Standard hours: As it was always 80 for all employees this columns was useless.
Over18 : Yes for all
Employee number: This column was not usefull for what we are looking for and could have confused the ANN.
The Attrition as been moved to the first column to make the slicing easier later on.

In [4]:
dataset = dataset[['Attrition',
                   'Age',
                   'BusinessTravel',
                   'DailyRate',
                   'Department',
                   'DistanceFromHome',
                   'Education',
                   'EducationField',
                   'EmployeeCount',
                   'EnvironmentSatisfaction',
                   'Gender',
                   'HourlyRate',
                   'JobInvolvement',
                   'JobLevel',
                   'JobRole',
                   'JobSatisfaction',
                   'MaritalStatus',
                   'MonthlyIncome',
                   'MonthlyRate',
                   'NumCompaniesWorked',
                   'OverTime',
                   'PercentSalaryHike',
                   'PerformanceRating',
                   'RelationshipSatisfaction',
                   'StockOptionLevel',
                   'TotalWorkingYears',
                   'TrainingTimesLastYear',
                   'WorkLifeBalance',
                   'YearsAtCompany',
                   'YearsInCurrentRole',
                   'YearsSinceLastPromotion',
                   'YearsWithCurrManager']]

## Adding some more derived information

Playing with the data we can add some more information to help the network.
We don't need to be expert in the field in Deep Learning, but providing some ratios and extra information can help the network to converge faster.

In [5]:
dataset['JobInvolment_On_Salary']= dataset['JobInvolvement'] / dataset['MonthlyIncome'] * 1000
dataset['MarriedAndBad_Worklife_Balance'] = np.where(dataset['MaritalStatus']=='Married', 
                                               dataset['WorkLifeBalance']-2,
                                               dataset['WorkLifeBalance']+1)
dataset['DistanceFromHome_rootedTo_JobSatisfaction'] = dataset['DistanceFromHome']**(1/dataset['JobSatisfaction'])
dataset['TotalJobSatisfaction'] = dataset['EnvironmentSatisfaction'] + dataset['JobSatisfaction'] + dataset['RelationshipSatisfaction'] 
dataset['OldLowEmployeeTendToStay'] = dataset['YearsAtCompany'] / dataset['JobLevel']
dataset['Mothers'] = np.where((dataset['Gender']=='Female') & (dataset['Age']>=36), 1,0)
dataset['Rate'] = dataset['DailyRate'] * 20 + dataset['HourlyRate'] * 8 * 20 + dataset['MonthlyRate']
dataset['RateExtended'] = dataset['Rate'] * (8 - dataset['JobSatisfaction'] - dataset['EnvironmentSatisfaction'])

## Seperating the data from the labels

In [6]:
X = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

## Encoding the data and the labels

Neural network only understand numbers. We need to transfer the columns with strings into numbers.

Create categories.
Male as 0 Female as 1

In [7]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_3 = LabelEncoder()
X[:, 3] = labelencoder_X_3.fit_transform(X[:, 3])
labelencoder_X_6= LabelEncoder()
X[:, 6] = labelencoder_X_6.fit_transform(X[:, 6])
labelencoder_X_9= LabelEncoder()
X[:, 9] = labelencoder_X_9.fit_transform(X[:, 9])
labelencoder_X_13= LabelEncoder()
X[:, 13] = labelencoder_X_13.fit_transform(X[:, 13])
labelencoder_X_15= LabelEncoder()
X[:, 15] = labelencoder_X_15.fit_transform(X[:, 15])
labelencoder_X_19= LabelEncoder()
X[:, 19] = labelencoder_X_19.fit_transform(X[:, 19])
X = X.astype(float)
labelencoder_y= LabelEncoder()
y = labelencoder_y.fit_transform(y)

## Dummy variable and dummy trap
Here we will create dummy variable for all the categorical data we just encode. (Only if there is more than 2 categories).

We are doing this because if we leave Single as 0, Married as 1 and Divorced as 2, the network would understand that divorced > married, which doesn't make any sense.


Yet we do not want to fall in the dummy variable trap and we will be removing the first column of each of those dummy variables.

Why?

Because if we have 1 0 0 for a Single Person now. We could guess it is single even if we had removed the first column: 0 0 (not divorced, not married --> single). This way we can remove some duplicated features on each OneHotEncoding

In [8]:
#no dummy trap
onehotencoder1 = OneHotEncoder(categorical_features = [1])
X = onehotencoder1.fit_transform(X).toarray()
X = X[:,1:]
onehotencoder3 = OneHotEncoder(categorical_features = [4])
X = onehotencoder3.fit_transform(X).toarray()
X = X[:,1:]
onehotencoder6 = OneHotEncoder(categorical_features = [8])
X = onehotencoder6.fit_transform(X).toarray()
X = X[:,1:]
onehotencoder13 = OneHotEncoder(categorical_features = [19])
X = onehotencoder13.fit_transform(X).toarray()
X = X[:,1:]
onehotencoder15 = OneHotEncoder(categorical_features = [28])
X = onehotencoder15.fit_transform(X).toarray()
X = X[:,1:]

## Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Scaling the features

We want to have all the features on a similar scale.

This helps in the computational aspect, but also helps the network not having some features that look much more important than the others by size

In [10]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Hyperparameters

To avoid overfitting such a tiny dataset we will use dropout (randomly putting "off" 10% of the neurons to help them become more independent)

All those parameters are the "winners" of a GridSearch I performed locally.

In [11]:
dropout = 0.1
epochs = 100
batch_size = 30
optimizer = 'adam'
k = 20

## Training the Neural Network, using a K Fold Cross Validation

For initializing the weight we will use a truncated normal distribution.

As we want a probability for output we will use a sigmoid activation function on the output layer.

Because we are working on a categorization problem with only 2 category we will calculate our loss with binary cross entropy.

We will use a 10 Fold Cross validation here: the validation data will be pick randomly K times and we will train the network 10 times on each of those training-validation set.

In [12]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(16, kernel_initializer="truncated_normal", activation = 'relu', input_shape = (X.shape[1],)))
    classifier.add(Dropout(dropout))
    classifier.add(Dense(1, kernel_initializer="truncated_normal", activation = 'sigmoid', )) #outputlayer
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ["accuracy"])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = batch_size, epochs = epochs, verbose=1)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 30)
max = accuracies.max()

Using TensorFlow backend.


Epoch 1/100
1136/1136 [==============================] - 301s - loss: 0.6706 - acc: 0.7764   
Epoch 2/100
1136/1136 [==============================] - 1s - loss: 0.6030 - acc: 0.8389     - ETA: 0s - loss: 0.6286 - a
Epoch 3/100
1136/1136 [==============================] - 1s - loss: 0.5182 - acc: 0.8389     
Epoch 4/100
1136/1136 [==============================] - 0s - loss: 0.4497 - acc: 0.8389     
Epoch 5/100
1136/1136 [==============================] - 0s - loss: 0.4078 - acc: 0.8389     
Epoch 6/100
1136/1136 [==============================] - 0s - loss: 0.3841 - acc: 0.8389     
Epoch 7/100
1136/1136 [==============================] - 0s - loss: 0.3628 - acc: 0.8389     
Epoch 8/100
1136/1136 [==============================] - 0s - loss: 0.3549 - acc: 0.8398     
Epoch 9/100
1136/1136 [==============================] - 0s - loss: 0.3397 - acc: 0.8460     
Epoch 10/100
1136/1136 [==============================] - 0s - loss: 0.3293 - acc: 0.8636     
Epoch 11/100
1136/1136 [=======

1136/1136 [==============================] - 0s - loss: 0.1830 - acc: 0.9243     
Epoch 86/100
1136/1136 [==============================] - 0s - loss: 0.1818 - acc: 0.9269     
Epoch 87/100
1136/1136 [==============================] - 0s - loss: 0.1816 - acc: 0.9278     
Epoch 88/100
1136/1136 [==============================] - 0s - loss: 0.1737 - acc: 0.9261     
Epoch 89/100
1136/1136 [==============================] - 0s - loss: 0.1915 - acc: 0.9208     
Epoch 90/100
1136/1136 [==============================] - 0s - loss: 0.1813 - acc: 0.9243     
Epoch 91/100
1136/1136 [==============================] - 0s - loss: 0.1826 - acc: 0.9252     
Epoch 92/100
1136/1136 [==============================] - 0s - loss: 0.1864 - acc: 0.9155     
Epoch 93/100
1136/1136 [==============================] - 0s - loss: 0.1760 - acc: 0.9287     
Epoch 94/100
1136/1136 [==============================] - 0s - loss: 0.1764 - acc: 0.9278     
Epoch 95/100
1136/1136 [==============================] - 0s - 

1136/1136 [==============================] - 0s - loss: 0.1985 - acc: 0.9146     
Epoch 71/100
1136/1136 [==============================] - 0s - loss: 0.1934 - acc: 0.9190     
Epoch 72/100
1136/1136 [==============================] - 0s - loss: 0.2004 - acc: 0.9199     
Epoch 73/100
1136/1136 [==============================] - 0s - loss: 0.1921 - acc: 0.9217     
Epoch 74/100
1136/1136 [==============================] - 0s - loss: 0.2006 - acc: 0.9164     
Epoch 75/100
1136/1136 [==============================] - 0s - loss: 0.1935 - acc: 0.9146     
Epoch 76/100
1136/1136 [==============================] - 0s - loss: 0.1906 - acc: 0.9217     
Epoch 77/100
1136/1136 [==============================] - 0s - loss: 0.1903 - acc: 0.9261     
Epoch 78/100
1136/1136 [==============================] - 0s - loss: 0.1907 - acc: 0.9225     
Epoch 79/100
1136/1136 [==============================] - 0s - loss: 0.1911 - acc: 0.9234     
Epoch 80/100
1136/1136 [==============================] - 0s - 

1136/1136 [==============================] - 0s - loss: 0.2506 - acc: 0.8961     
Epoch 54/100
1136/1136 [==============================] - 0s - loss: 0.2490 - acc: 0.9032     
Epoch 55/100
1136/1136 [==============================] - 0s - loss: 0.2519 - acc: 0.8988     
Epoch 56/100
1136/1136 [==============================] - 0s - loss: 0.2481 - acc: 0.9040     
Epoch 57/100
1136/1136 [==============================] - 0s - loss: 0.2499 - acc: 0.9040     
Epoch 58/100
1136/1136 [==============================] - 0s - loss: 0.2449 - acc: 0.9085     
Epoch 59/100
1136/1136 [==============================] - 0s - loss: 0.2421 - acc: 0.9067     
Epoch 60/100
1136/1136 [==============================] - 0s - loss: 0.2442 - acc: 0.9085     
Epoch 61/100
1136/1136 [==============================] - 0s - loss: 0.2405 - acc: 0.9023     
Epoch 62/100
1136/1136 [==============================] - 0s - loss: 0.2418 - acc: 0.9093     
Epoch 63/100
1136/1136 [==============================] - 0s - 

...

1137/1137 [==============================] - 0s - loss: 0.2502 - acc: 0.9068     
Epoch 28/100
1137/1137 [==============================] - 0s - loss: 0.2544 - acc: 0.8953     
Epoch 29/100
1137/1137 [==============================] - 0s - loss: 0.2471 - acc: 0.8989     
Epoch 30/100
1137/1137 [==============================] - 0s - loss: 0.2421 - acc: 0.9015     
Epoch 31/100
1137/1137 [==============================] - 0s - loss: 0.2454 - acc: 0.8971     
Epoch 32/100
1137/1137 [==============================] - 0s - loss: 0.2441 - acc: 0.8989     
Epoch 33/100
1137/1137 [==============================] - 0s - loss: 0.2397 - acc: 0.9041     
Epoch 34/100
1137/1137 [==============================] - 0s - loss: 0.2398 - acc: 0.8997     
Epoch 35/100
1137/1137 [==============================] - 0s - loss: 0.2349 - acc: 0.9059     
Epoch 36/100
1137/1137 [==============================] - 0s - loss: 0.2265 - acc: 0.9129     
Epoch 37/100
1137/1137 [==============================] - 0s - 

1137/1137 [==============================] - 0s - loss: 0.3125 - acc: 0.8734     
Epoch 12/100
1137/1137 [==============================] - 0s - loss: 0.3017 - acc: 0.8813     
Epoch 13/100
1137/1137 [==============================] - 0s - loss: 0.2956 - acc: 0.8839     
Epoch 14/100
1137/1137 [==============================] - 1s - loss: 0.2914 - acc: 0.8909     
Epoch 15/100
1137/1137 [==============================] - 0s - loss: 0.2860 - acc: 0.8901     
Epoch 16/100
1137/1137 [==============================] - 0s - loss: 0.2825 - acc: 0.8883     
Epoch 17/100
1137/1137 [==============================] - 0s - loss: 0.2816 - acc: 0.8857     
Epoch 18/100
1137/1137 [==============================] - 0s - loss: 0.2752 - acc: 0.8892     
Epoch 19/100
1137/1137 [==============================] - 0s - loss: 0.2694 - acc: 0.8927     
Epoch 20/100
1137/1137 [==============================] - 0s - loss: 0.2646 - acc: 0.8997     
Epoch 21/100
1137/1137 [==============================] - 0s - 

1137/1137 [==============================] - 0s - loss: 0.1584 - acc: 0.9323     
Epoch 98/100
1137/1137 [==============================] - 0s - loss: 0.1704 - acc: 0.9296     
Epoch 99/100
1137/1137 [==============================] - 0s - loss: 0.1697 - acc: 0.9358     
Epoch 100/100
1137/1137 [==============================] - 3s - loss: 0.6656 - acc: 0.7986     
Epoch 2/100
1137/1137 [==============================] - 0s - loss: 0.5967 - acc: 0.8391     
Epoch 3/100
1137/1137 [==============================] - 0s - loss: 0.5156 - acc: 0.8391     
Epoch 4/100
1137/1137 [==============================] - 0s - loss: 0.4492 - acc: 0.8391     
Epoch 5/100
1137/1137 [==============================] - 0s - loss: 0.4086 - acc: 0.8391     
Epoch 6/100
1137/1137 [==============================] - 0s - loss: 0.3810 - acc: 0.8391     
Epoch 7/100
1137/1137 [==============================] - 0s - loss: 0.3672 - acc: 0.8391     - ETA: 0s - loss: 0.3748 - a
Epoch 8/100
1137/1137 [=================

1137/1137 [==============================] - 0s - loss: 0.1847 - acc: 0.9252     
Epoch 83/100
1137/1137 [==============================] - 0s - loss: 0.1843 - acc: 0.9261     
Epoch 84/100
1137/1137 [==============================] - 0s - loss: 0.1865 - acc: 0.9261     
Epoch 85/100
1137/1137 [==============================] - 0s - loss: 0.1934 - acc: 0.9191     
Epoch 86/100
1137/1137 [==============================] - 0s - loss: 0.1760 - acc: 0.9288     
Epoch 87/100
1137/1137 [==============================] - 0s - loss: 0.1819 - acc: 0.9270     
Epoch 88/100
1137/1137 [==============================] - 0s - loss: 0.1758 - acc: 0.9279     
Epoch 89/100
1137/1137 [==============================] - 0s - loss: 0.1752 - acc: 0.9288     
Epoch 90/100
1137/1137 [==============================] - 0s - loss: 0.1832 - acc: 0.9244     
Epoch 91/100
1137/1137 [==============================] - 0s - loss: 0.1841 - acc: 0.9252     
Epoch 92/100
1137/1137 [==============================] - 0s - 

1137/1137 [==============================] - 0s - loss: 0.1891 - acc: 0.9235     
Epoch 68/100
1137/1137 [==============================] - 0s - loss: 0.2039 - acc: 0.9112     
Epoch 69/100
1137/1137 [==============================] - 0s - loss: 0.1957 - acc: 0.9129     
Epoch 70/100
1137/1137 [==============================] - 0s - loss: 0.1933 - acc: 0.9182     
Epoch 71/100
1137/1137 [==============================] - 0s - loss: 0.1843 - acc: 0.9244     
Epoch 72/100
1137/1137 [==============================] - 0s - loss: 0.1955 - acc: 0.9173     
Epoch 73/100
1137/1137 [==============================] - 0s - loss: 0.1907 - acc: 0.9226     
Epoch 74/100
1137/1137 [==============================] - 0s - loss: 0.1765 - acc: 0.9288     
Epoch 75/100
1137/1137 [==============================] - 0s - loss: 0.1975 - acc: 0.9138     
Epoch 76/100
1137/1137 [==============================] - 0s - loss: 0.1822 - acc: 0.9173     
Epoch 77/100
1137/1137 [==============================] - 0s - 

In [13]:
print("Best accuracy: ",max)

Best accuracy:  0.948717997624
